In [1]:
from app.loader import add_docs_to_knowledge_base
from app.db import create_vectors_index, count_vectors
from IPython import display

In [2]:
await create_vectors_index()

Index idx:vectors created successfully


In [3]:
await add_docs_to_knowledge_base()

Loading documents


100%|██████████| 1/1 [00:04<00:00,  4.14s/it]


Loaded 1 PDF documents

Splitting documents into chunks
Inception.pdf: 127 chunks

Total chunks: 127
Min chunk size: 351 tokens
Max chunk size: 512 tokens
Average chunk size: 496 tokens

Embedding chunks


100%|██████████| 127/127 [00:02<00:00, 45.94it/s]


Adding chunks to vector DB


In [4]:
total_vectors = await count_vectors()
print(f'Total vectors: {total_vectors}')

Total vectors: 127


In [10]:
import json
from time import time
from uuid import uuid4
from IPython.display import display, JSON
from redis.asyncio import Redis
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from redis.commands.json.path import Path
from app.config import settings
from app.db import create_chat_index, create_chat, add_chat_messages, get_chat_messages, get_all_chats, delete_chats

r = Redis(host=settings.REDIS_HOST, port=settings.REDIS_PORT)

In [3]:
await create_chat_index()

Error creating index idx:chat: Index already exists


In [2]:
res = await r.ft('idx:chat').search(Query('*'))
for doc in res.docs:
    display(JSON(json.loads(doc.json)))

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

In [5]:
chat_id = str(uuid4())[:8]
created = int(time())
chat = await create_chat(chat_id, created)
chat

{'id': '376606d1', 'created': 1722970907, 'messages': []}

In [6]:
messages = [
    {'role': 'user', 'content': 'How can you help me?', 'created': int(time())},
    {'role': 'assistant', 'content': 'I am a helpful assistant', 'created': int(time())}
]
await add_chat_messages(chat_id='787acc90', messages=messages)

In [7]:
await get_chat_messages(chat_id='787acc90', last_n=2)

[{'role': 'user', 'content': 'How can you help me?', 'created': 1722970907},
 {'role': 'assistant',
  'content': 'I am a helpful assistant',
  'created': 1722970907}]

In [13]:
await get_all_chats()

[{'id': '1931cbac',
  'created': 1723137027,
  'messages': [{'role': 'user', 'content': 'Hello there'},
   {'role': 'assistant', 'content': 'Hello, how can I help you?'}]}]

In [35]:
res = await r.ft('idx:chat').search(Query('*').return_field("$.id", as_field="chat_id"))
print([doc.chat_id for doc in res.docs])
# await delete_chats(*[doc.chat_id for doc in res.docs])

[]


In [12]:
await add_chat_messages('1931cbac', [
    {'role': 'user', 'content': 'Hello there'},
    {'role': 'assistant', 'content': 'Hello, how can I help you?'}
])